## Datetime Partitioning

**Author**: Thodoris Petropoulos

**Label**: Modeling Options

### Scope

The scope of this notebook is to provide instructions on how to do datetime partitioning using the Python API.

### Background

When dividing your data for model training and validation, DataRobot will randomly choose a set of rows from your dataset to assign amongst different cross validation folds. This will verify that you have not overfit your model to the training set and that the model can perform well on new data. However when your data has an intrinsic time based component, then you have to be even more careful about target leakage.  DataRobot now posseses datetime partitioning which will be diligent within model training & validation to guard against this, but you should always use your domain exerptise to evaluate your features prior to modeling.

Let’s look at how we would frame a problem with a time component within DataRobot. This project basically simulated what you would get if you tried "Out of Time Validation" within DataRobot interface which is **not the same as Time Series projects**, even though the way we define backtests is very similar.

### Requirements

- Python version 3.7.3
-  DataRobot API version 2.20.0. 
Small adjustments might be needed depending on the Python version and DataRobot API version you are using.

Full documentation of the Python package can be found here: https://datarobot-public-api-client.readthedocs-hosted.com

It is assumed you already have a DataRobot <code>Project</code> object and a DataRobot <code>Model </code> object.

#### Import Libraries

In [ ]:
import datarobot as dr

#### Running a DataRobot Project with a datetime partition
The durations need to be specified in string using the format of the <code>dr.helpers.partitioning_methods.construct_duration_string()</code> method.

In [ ]:
spec = dr.DatetimePartitioningSpecification(datetime_partition_column = 'Date',
                                            holdout_start_date=datetime(2017,1,2),
                                            holdout_duration='P1Y0M0DT0H0M0S',
                                            number_of_backtests = 2,
                                            use_time_series = False)

We took advantage of DataRobot’s automated partition date selection after we specified the number of backtests to use. DataRobot allows further control, where we can further specify the validation start date as well as duration. Let’s look at an example below.

#### Create Backtest Specifications
The below method would work for both Time Series and Out of Time Validation projects. Currently, we have specified that <code>use_time_series = False</code> in the <code>dr.DatetimePartitioningSpecification()</code> method so this would be initiated as an OTV project.

In [ ]:
# Dates are not project specific but rather example dates
spec.backtests=[dr.BacktestSpecification(0,gap_duration = 'P0Y0M0DT0H0M0S',
                                         validation_start_date = datetime(2016,1,2), 
                                         validation_duration = 'P1Y0M0DT0H0M0S'),


                dr.BacktestSpecification(1,gap_duration = 'P0Y0M0DT0H0M0S',
                                         validation_start_date = datetime(2015,1,2), 
                                         validation_duration = 'P1Y0M0DT0H0M0S')]

#To start the project
project = dr.Project.create(sourcedata = df, project_name = 'Project Name')
project.set_target('target_column',partitioning_method = spec)

The above methods will change the backtest specification for the first and second backtests.